In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math as math
import seaborn as sns

In [2]:
#data set from kaggle: https://www.kaggle.com/competitions/grupo-bimbo-inventory-demand/data

#office
bgtrain = pd.read_csv("../../Desktop/work/data science/project/bakery data/train.csv")
clients = pd.read_csv("../../Desktop/work/data science/project/bakery data/cliente_tabla.csv")
town_state = pd.read_csv("../../Desktop/work/data science/project/bakery data/town_state.csv")
product_name = pd.read_csv("../../Desktop/work/data science/project/bakery data/producto_tabla.csv")
bgtest = pd.read_csv("../../Desktop/work/data science/project/bakery data/test.csv")
sample_submission = pd.read_csv("../../Desktop/work/data science/project/bakery data/sample_submission.csv")

#submission must have 6999251 rows, i.e. the length of the test data set

In [154]:
len(bgtest)

6999251

In [3]:
bgtrain = bgtrain.rename(columns={'Semana': 'Week_num',
                                  'Agencia_ID': 'Sales_Depot_ID',
                                  'Canal_ID': 'Sales_Channel_ID',
                                  'Ruta_SAK': 'Route_ID',
                                  'Cliente_ID': 'Client_ID',
                                  'Venta_uni_hoy': 'Sales_unit_this_week',
                                  'Venta_hoy': 'Sales_this_week',
                                  'Dev_uni_proxima': 'Returns_unit_next_week',
                                  'Dev_proxima': 'Returns_next_week',
                                  'Demanda_uni_equil': 'Adjusted_demand',
                                  'Producto_ID': 'Product_ID'})

bgtest = bgtest.rename(columns={'Semana': 'Week_num',
                                  'Agencia_ID': 'Sales_Depot_ID',
                                  'Canal_ID': 'Sales_Channel_ID',
                                  'Ruta_SAK': 'Route_ID',
                                  'Cliente_ID': 'Client_ID',
                                  'Venta_uni_hoy': 'Sales_unit_this_week',
                                  'Venta_hoy': 'Sales_this_week',
                                  'Dev_uni_proxima': 'Returns_unit_next_week',
                                  'Dev_proxima': 'Returns_next_week',
                                  'Demanda_uni_equil': 'Adjusted_demand',
                                  'Producto_ID': 'Product_ID'})

The most naive guess is to use the same adjusted demand from week 9.  If the client or product is not present in week 9, set adjusted demand to 7.

In [7]:
#this reduces the entries from 75 million to 10 million

simple_bgtrain = pd.DataFrame({'Client_ID': bgtrain[bgtrain['Week_num'] == 9]['Client_ID'],
                               'Product_ID': bgtrain[bgtrain['Week_num'] == 9]['Product_ID'],
                               'Adjusted_demand': bgtrain[bgtrain['Week_num'] == 9]['Adjusted_demand']})

In [27]:
simple_bgtest = pd.DataFrame({'Client_ID': bgtest['Client_ID'],
                              'Product_ID': bgtest['Product_ID']})

In [8]:
df = pd.DataFrame({'id': np.arange(len(bgtest)),
                   'Demanda_uni_equil': 7*np.ones(len(bgtest)).astype(int)})

In [14]:
simple_bgtrain[(simple_bgtrain['Client_ID'] == bgtest['Client_ID'].iloc[2]) & (simple_bgtrain['Product_ID'] == bgtest['Product_ID'].iloc[2])]['Adjusted_demand']

71784635    2
Name: Adjusted_demand, dtype: int64

In [70]:
for i in range(2000000):
    if len(simple_bgtrain[(simple_bgtrain['Client_ID'] == simple_bgtest['Client_ID'].iloc[i]) & (simple_bgtrain['Product_ID'] == simple_bgtest['Product_ID'].iloc[i])]) != 0:
        df.loc[i, 'Demanda_uni_equil'] = simple_bgtrain[(simple_bgtrain['Client_ID'] == simple_bgtest['Client_ID'].iloc[i]) & (simple_bgtrain['Product_ID'] == simple_bgtest['Product_ID'].iloc[i])]['Adjusted_demand'].iloc[0]

In [ ]:
for i in np.arange(4000000,6999251,1):
    if len(simple_bgtrain[(simple_bgtrain['Client_ID'] == simple_bgtest['Client_ID'].iloc[i]) & (simple_bgtrain['Product_ID'] == simple_bgtest['Product_ID'].iloc[i])]) != 0:
        df.loc[i, 'Demanda_uni_equil'] = simple_bgtrain[(simple_bgtrain['Client_ID'] == simple_bgtest['Client_ID'].iloc[i]) & (simple_bgtrain['Product_ID'] == simple_bgtest['Product_ID'].iloc[i])]['Adjusted_demand'].iloc[0]

In [ ]:
df.to_csv('simple_prediction.csv', index=False)

Improve the simple_prediction.csv file by making guesses for new clients and/or products by using averages.

In [2]:
#data set from kaggle: https://www.kaggle.com/competitions/grupo-bimbo-inventory-demand/data

#office
train = pd.read_csv("train.csv", usecols=['Semana', 'Producto_ID', 'Cliente_ID', 'Demanda_uni_equil'])
test = pd.read_csv("test.csv", usecols=['Semana', 'Producto_ID', 'Cliente_ID', 'id'])

train = train.rename(columns={'Semana': 'Week_num',
                                  'Agencia_ID': 'Sales_Depot_ID',
                                  'Canal_ID': 'Sales_Channel_ID',
                                  'Ruta_SAK': 'Route_ID',
                                  'Cliente_ID': 'Client_ID',
                                  'Venta_uni_hoy': 'Sales_unit_this_week',
                                  'Venta_hoy': 'Sales_this_week',
                                  'Dev_uni_proxima': 'Returns_unit_next_week',
                                  'Dev_proxima': 'Returns_next_week',
                                  'Demanda_uni_equil': 'adjusted_demand',
                                  'Producto_ID': 'Product_ID'})

test = test.rename(columns={'Semana': 'Week_num',
                                  'Agencia_ID': 'Sales_Depot_ID',
                                  'Canal_ID': 'Sales_Channel_ID',
                                  'Ruta_SAK': 'Route_ID',
                                  'Cliente_ID': 'Client_ID',
                                  'Venta_uni_hoy': 'Sales_unit_this_week',
                                  'Venta_hoy': 'Sales_this_week',
                                  'Dev_uni_proxima': 'Returns_unit_next_week',
                                  'Dev_proxima': 'Returns_next_week',
                                  'Demanda_uni_equil': 'adjusted_demand',
                                  'Producto_ID': 'Product_ID'})



#set a unique id for each client, product combination (thanks Gemini)
combined_df = pd.concat([train,test])
combined_df['ID'] = combined_df.groupby(['Client_ID', 'Product_ID']).ngroup()

train = combined_df.iloc[:len(train)].copy()
test = combined_df.iloc[len(train):].copy()

del combined_df


train = train.drop(columns='id')
train['adjusted_demand'] = train['adjusted_demand'].astype(int)
train = train.sort_values(by=['ID', 'Week_num']).reset_index(drop=True)

test = test.drop(columns='adjusted_demand')
test['id'] = test['id'].astype(int)
test = test.sort_values(by=['ID', 'Week_num']).reset_index(drop=True)

In [3]:
#create a dataframe of aggregate statistics for each client
testagg = train.groupby('Client_ID').agg({'Product_ID':'nunique', 'adjusted_demand':['mean', 'median', 'std', 'min', 'max']})
testagg = testagg.reset_index()

client_stats = pd.DataFrame()

client_stats['Client_ID'] = train['Client_ID'].unique()
client_stats['Products'] = testagg['Product_ID']['nunique']
client_stats['adj_dem_mean'] = testagg['adjusted_demand']['mean'].round(2)
client_stats['adj_dem_median'] = testagg['adjusted_demand']['median'].astype(int)
client_stats['adj_dem_std'] = testagg['adjusted_demand']['std'].round(2)
client_stats['adj_dem_min'] = testagg['adjusted_demand']['min']
client_stats['adj_dem_max'] = testagg['adjusted_demand']['max']



#create a dataframe of aggregate statistics for each product
del testagg
testagg = train.sort_values(by=['Product_ID']).groupby('Product_ID').agg({'Client_ID':'nunique', 'adjusted_demand':['mean', 'median', 'std', 'min', 'max']})
testagg = testagg.reset_index()

product_stats = pd.DataFrame()

product_stats['Product_ID'] = train.sort_values(by=['Product_ID'])['Product_ID'].unique()
product_stats['Clients'] = testagg['Client_ID']['nunique']
product_stats['adj_dem_mean'] = testagg['adjusted_demand']['mean'].round(2)
product_stats['adj_dem_median'] = testagg['adjusted_demand']['median'].astype(int)
product_stats['adj_dem_std'] = testagg['adjusted_demand']['std'].round(2)
product_stats['adj_dem_min'] = testagg['adjusted_demand']['min']
product_stats['adj_dem_max'] = testagg['adjusted_demand']['max']

del testagg

client_stats.head()

,Client_ID,Products,adj_dem_mean,adj_dem_median,adj_dem_std,adj_dem_min,adj_dem_max
0,26,51,28.10,20,26.62,0,132
1,60,34,188.83,130,228.27,11,2304
2,65,112,109.10,60,140.54,0,1669
3,101,4,81.60,28,86.40,20,280
4,105,72,50.36,26,57.83,2,459


In [23]:
product_stats.head()

,Product_ID,Clients,adj_dem_mean,adj_dem_median,adj_dem_std,adj_dem_min,adj_dem_max
0,41,15,367.44,70,532.94,0,2064
1,53,45,291.10,119,409.19,0,1919
2,72,67417,4.90,4,5.16,0,659
3,73,59643,3.19,2,7.71,0,450
4,100,40,8.81,3,11.83,0,52


In [4]:
testID = test['ID'].unique().tolist()
trainID = train['ID'].unique().tolist()
commonID = list(set(testID).intersection(set(trainID)))
newID = list(set(testID) - set(trainID))

In [9]:
#verify that newID and commonID have the right number of elements

print(len(newID)+len(commonID))

print(len(test['ID'].unique()))

6225450
6225450


In [ ]:
#for a old client buying an old product that hasn't been purchased in the first 9 weeks, use aggregate stats for the client and product
#for a new client but old product, predict using the aggregate product stats
#for a new product but old client, predict using the aggregate client stats
#for a new product and client, set adjusted demand to 3

In [57]:
newdf = test[test['ID'].isin(newID)].copy()

newdf.loc[:, 'Old_Product'] = newdf['Product_ID'].isin(train['Product_ID'].unique().tolist())
newdf.loc[:, 'Old_Client'] = newdf['Client_ID'].isin(train['Client_ID'].unique().tolist())
newdf.loc[:, 'adjusted_demand'] = 7*np.ones(len(newdf)).astype(int)
newdf['Client_mean'] = np.zeros(len(newdf))
newdf['Product_mean'] = np.zeros(len(newdf))

#set the mean of new clients or new products to NaN
newdf.loc[newdf['Old_Client'] == False, 'Client_mean'] = np.nan
newdf.loc[newdf['Old_Product'] == False, 'Product_mean'] = np.nan

#assign Product means (takes roughly 5 seconds)
pid = newdf[newdf['Old_Product'] == True]['Product_ID'].unique().tolist()

for i in range(len(pid)):
    newdf.loc[newdf['Product_ID'] == pid[i], 'Product_mean'] = product_stats[product_stats['Product_ID'] == pid[i]]['adj_dem_mean'].iloc[0]

newdf.head()

,Week_num,Client_ID,Product_ID,id,ID,Old_Product,Old_Client,adjusted_demand,Client_mean,Product_mean
1,11,26,31520,4728674,6,True,True,7,0.0,28.06
13,10,26,43388,4837077,46,True,True,7,0.0,16.33
59,10,65,35305,592764,138,True,True,7,0.0,9.18
131,11,107,4767,460185,297,True,True,7,0.0,7.23
151,11,107,43027,2186063,334,True,True,7,0.0,49.05


In [18]:
newdf['Old_Client'].value_counts()

Old_Client
True     1335159
False      30054
Name: count, dtype: int64

In [19]:
newdf['Old_Product'].value_counts()

Old_Product
True     1339573
False      25640
Name: count, dtype: int64

In [58]:
#assign Client means (takes 45 min)
cid = newdf[newdf['Old_Client'] == True]['Client_ID'].unique().tolist()

for i in range(len(cid)):
    newdf.loc[newdf['Client_ID'] == cid[i], 'Client_mean'] = client_stats[client_stats['Client_ID'] == cid[i]]['adj_dem_mean'].iloc[0]


newdf.to_csv("non_seven_guesses.csv")

In [5]:
newdf = pd.read_csv("non_seven_guesses.csv")
newdf = newdf.drop('Unnamed: 0', axis=1)

newdf.head()

,Week_num,Client_ID,Product_ID,id,ID,Old_Product,Old_Client,adjusted_demand,Client_mean,Product_mean
0,11,26,31520,4728674,6,True,True,7,28.10,28.06
1,10,26,43388,4837077,46,True,True,7,28.10,16.33
2,10,65,35305,592764,138,True,True,7,109.10,9.18
3,11,107,4767,460185,297,True,True,7,23.45,7.23
4,11,107,43027,2186063,334,True,True,7,23.45,49.05


In [33]:
#if old client is new to buying old product, set the adjusted demand as the average of client, product mean
newdf.loc[(newdf['Old_Client'] == True) & (newdf['Old_Product'] == True), 'adjusted_demand'] = ((1/2)*(newdf[(newdf['Old_Client'] == True) & (newdf['Old_Product'] == True)]['Client_mean']+newdf[(newdf['Old_Client'] == True) & (newdf['Old_Product'] == True)]['Product_mean'])).round(0)

#if old client is buying new product, set the adjusted demand as the ceiling of client mean
newdf.loc[(newdf['Old_Client'] == True) & (newdf['Old_Product'] == False), 'adjusted_demand'] = np.ceil(newdf[(newdf['Old_Client'] == True) & (newdf['Old_Product'] == False)]['Client_mean'])

#if new client is buying old product, set the adjusted demand as the floor of product mean
newdf.loc[(newdf['Old_Client'] == False) & (newdf['Old_Product'] == True), 'adjusted_demand'] = np.ceil(newdf[(newdf['Old_Client'] == False) & (newdf['Old_Product'] == True)]['Product_mean'])

#if new client is buying new product, set the adjusted demand to 3
newdf.loc[(newdf['Old_Client'] == False) & (newdf['Old_Product'] == False), 'adjusted_demand'] = 7

newdf['adjusted_demand'] = newdf['adjusted_demand'].astype(int)

newdf.head()

,Week_num,Client_ID,Product_ID,id,ID,Old_Product,Old_Client,adjusted_demand,Client_mean,Product_mean
0,11,26,31520,4728674,6,True,True,28,28.10,28.06
1,10,26,43388,4837077,46,True,True,22,28.10,16.33
2,10,65,35305,592764,138,True,True,59,109.10,9.18
3,11,107,4767,460185,297,True,True,15,23.45,7.23
4,11,107,43027,2186063,334,True,True,36,23.45,49.05


In [4]:
spdf = pd.read_csv("simple_prediction.csv")

spdf.head()

,id,Demanda_uni_equil
0,0,4
1,1,7
2,2,2
3,3,7
4,4,7


In [5]:
newdf = newdf.sort_values(by=['id'])

adc = pd.DataFrame({'id': newdf['id'], 'adjusted_demand': newdf['adjusted_demand']})
adc.set_index('id', inplace=True)

adc.head()

,adjusted_demand
id,
1,3
3,3
4,3
7,6
15,6


In [8]:
spdf.loc[spdf['id'].isin(newdf['id']), 'Demanda_uni_equil'] = adc['adjusted_demand']

spdf.head()

,id,Demanda_uni_equil
0,0,4
1,1,3
2,2,2
3,3,3
4,4,3


In [9]:
spdf['Demanda_uni_equil'].value_counts()

Demanda_uni_equil
7       1806215
2        739913
4        702008
3        642351
5        622293
         ...   
706           1
856           1
835           1
1081          1
567           1
Name: count, Length: 899, dtype: int64

In [10]:
spdf.to_csv("improved_simple_prediction_with3.csv", index=False)

Predict adjusted demand for all clients and products by weighted means.

In [7]:
olddf = test[test['ID'].isin(commonID)].copy()

#get client means
cidmapping = pd.Series(client_stats[client_stats['Client_ID'].isin(olddf['Client_ID'].unique().tolist())].set_index('Client_ID')['adj_dem_mean'], index=client_stats[client_stats['Client_ID'].isin(olddf['Client_ID'].unique().tolist())]['Client_ID']).to_dict()
olddf['Client_mean'] = olddf['Client_ID'].map(cidmapping)

#get product means
pidmapping = pd.Series(product_stats[product_stats['Product_ID'].isin(olddf['Product_ID'].unique().tolist())].set_index('Product_ID')['adj_dem_mean'], index=product_stats[product_stats['Product_ID'].isin(olddf['Product_ID'].unique().tolist())]['Product_ID']).to_dict()
olddf['Product_mean'] = olddf['Product_ID'].map(pidmapping)

olddf.head()

,Week_num,Client_ID,Product_ID,id,ID,Client_mean,Product_mean
0,10,26,31518,1569352,5,28.1,13.88
2,11,26,34206,1547831,13,28.1,22.89
3,10,26,34210,6667200,14,28.1,120.78
4,10,26,34785,1592616,18,28.1,86.23
5,11,26,34785,6825659,18,28.1,86.23


In [17]:
#verify that means were imported correctly

for i in np.random.choice(olddf['ID'].unique().tolist(), size=25):
    assert(olddf.loc[olddf['ID'] == i, 'Client_mean'].iloc[0] == train[train['Client_ID'] == train[train['ID'] == i]['Client_ID'].iloc[0]]['adjusted_demand'].mean().round(2))
    assert(olddf.loc[olddf['ID'] == i, 'Product_mean'].iloc[0] == train[train['Product_ID'] == train[train['ID'] == i]['Product_ID'].iloc[0]]['adjusted_demand'].mean().round(2))

In [21]:
for i in range(5):
    testdf = pd.DataFrame({f'{i}': [i+5]})
    testdf.to_csv(f"{i}_file.csv")

In [59]:
for p in np.linspace(0,1,11):
    oldad = pd.DataFrame({'id': olddf['id'], 'Demanda_uni_equil': np.ceil(p*olddf['Client_mean'] + (1-p)*olddf['Product_mean'])}).astype(int)

    newdf.loc[(newdf['Old_Client'] == True) & (newdf['Old_Product'] == True), 'adjusted_demand'] = (p*(newdf[(newdf['Old_Client'] == True) & (newdf['Old_Product'] == True)]['Client_mean']) + (1-p)*(newdf[(newdf['Old_Client'] == True) & (newdf['Old_Product'] == True)]['Product_mean'])).round(0)
    newad = pd.DataFrame({'id': newdf['id'], 'Demanda_uni_equil': newdf['adjusted_demand']})

    adc = pd.concat([oldad, newad])
    adc = adc.sort_values(by=['id']).reset_index(drop=True)

    adc.to_csv(f"{(10*p).round(0).astype(int)}_mean_prediction.csv", index=False)